# Test your code (shapes and values of returned arrays)

This notebook provides you some basic testing codes to check your implementation. Note that passing the tests given here doesn't necessarily mean that you have implemented everything correctly. If you cannot import edited .py files to this notebook, please consult E6 ipynb file for further instruction.

Once you pass all the tests, go ahead and play with E6, E7, and E8 ipynb files. If everything's implemented correctly, you should be able to train neural network models. You can verify that by observing the train and test errors, as well as graphs given in E7.

In [22]:
# Import necessary modules 
from model import NeuralNetwork
import numpy as np
import unittest

## Check shapes of numpy arrays
In this part, below cells of codes will test the shapes of returned arrays from functions you implemented.

### layer.py

In [23]:
from layer import FCLayer
in_dim, out_dim, num_sample = 2, 4, 10
x = np.ones((num_sample, in_dim))

nn = NeuralNetwork()
nn.add(FCLayer(in_dim, out_dim))
layer = nn.layers[0]

In [24]:
pred = nn.predict(x)
delta_n = np.random.randn(num_sample, out_dim)
delta = layer.backward(delta_n)

X is ok


In [25]:
assert pred.shape == (num_sample, out_dim), "Failed method: 'forward'\tIncorrect output shape."
assert delta.shape == (num_sample, in_dim), "Failed method: 'backward\tShape mismatch"
assert layer.weights.grad.shape == layer.weights.shape, "Failed method: 'backward'\tIncorrect dEdW shape"
assert layer.bias.grad.shape == layer.bias.shape, "Failed method: 'backward'\tIncorrect dEdb shape"

### activation.py

In [26]:
# Assuming you have correct layer.py
from utils import sigmoid_prime, sigmoid
from activation import Activation
in_dim, hidden_dim, out_dim, num_sample = 2, 8, 4, 10
x = np.ones((num_sample, in_dim))

nn = NeuralNetwork()
nn.add(FCLayer(in_dim, hidden_dim))
nn.add(Activation(sigmoid, sigmoid_prime))
nn.add(FCLayer(hidden_dim, out_dim))
sigmoid_layer = nn.layers[1]

In [27]:
nn.predict(x)
act = nn.layers[-1].input_data
delta_n = np.random.randn(num_sample, out_dim)
delta_n = nn.layers[-1].backward(delta_n)
delta = sigmoid_layer.backward(delta_n)

assert act.shape == (num_sample, hidden_dim), "Failed method: 'forward'\tIncorrect hidden layer shape."
assert delta.shape == (num_sample, hidden_dim), "Failed method: 'backward\tShape mismatch"

X is ok
X is ok


### loss.py

In [28]:
# Test MSE loss
from loss import MSELoss
num_samples = 10
pred = np.ones((num_samples, 1))
target = np.zeros((num_samples, 1))
loss = MSELoss()
mse = loss.loss(pred, target)
mse_diff = loss.diff_loss(pred, target)

assert mse.size == 1, "Failed method: 'loss' in MSELoss\tIncorrect loss shape."
assert mse_diff.shape == pred.shape, "Failed method: 'diff_loss' in MSELoss\tIncorrect diff_loss shape"

In [29]:
# Test Cross-entropy loss
from loss import CrossEntropyLoss
out_dim = 10
pred = np.random.rand(num_samples, out_dim)
target = np.eye(num_samples)
ce_loss = CrossEntropyLoss()
ce = ce_loss.loss(pred, target)
ce_diff = ce_loss.diff_loss(pred, target)

assert ce.size == 1, "Failed method: 'loss' in CrossEntropyLoss\tIncorrect loss shape"
assert ce_diff.shape == pred.shape, "Failed method: 'diff_loss' in CrossEntropyLoss\tIncorrect diff_loss shape"

## Unit test
Now, we will use unittest library to test the values of returned arrays. For each of the classes defined below, the *setUp* method defines some necessary attributes, while the returned values will be compared in the following methods starting with 'test_'. 

Should your code fail to pass one of the tests, you'll see where the error has occurred. Since you have passed the shape test, it may not be due to shape issues. Instead, try to double-check if your code is mathematically correct. 

#### FCLayer

In [30]:
class TestFullyConnectedLayer(unittest.TestCase):
    def setUp(self):
        import layer
        in_dim, out_dim = 2, 4
        self.layer = layer.FCLayer(in_dim, out_dim)
        self.layer.weights.value = np.array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318],
                                             [0.4236548 , 0.64589411, 0.43758721, 0.891773  ]])
        self.layer.bias.value = np.array([0.96366276, 0.38344152, 0.79172504, 0.52889492])[None, :]
        self.delta_n = np.array([[0.5507979 , 0.70814782, 0.29090474, 0.51082761],
                                 [0.89294695, 0.89629309, 0.12558531, 0.20724288]])
        def reset_input():
            self.layer.input_data = np.arange(4).reshape(2, 2)
        self.reset_input = reset_input
        
    def test_forward(self):
        X = np.arange(4).reshape(2, 2)
        output = self.layer.forward(X)
        answer = np.array([[1.38731756, 1.02933563, 1.22931225, 1.42066792],
                           [3.33225417, 3.75150259, 3.31001342, 4.29398029]])
        self.assertTrue(np.allclose(output, answer), "(FCLayer) forward method failed!")
        
    def test_backward(self):
        self.layer.input_data = np.random.randn(2, 2) # ignore this
        output = self.layer.backward(self.delta_n)
        answer = np.array([[1.26243321, 1.27357515],
                           [1.31970202, 1.19697982]])
        self.assertTrue(np.allclose(output, answer), "(FCLayer) backward method failed!")
    
    def test_gradient(self):
        self.reset_input()
        self.layer.backward(self.delta_n)
        dEdW = self.layer.weights.grad
        dEdb = self.layer.bias.grad
        w_answer = np.array([[1.7858939 , 1.79258618, 0.25117062, 0.41448576],
                             [3.22963875, 3.39702709, 0.66766067, 1.13255625]])
        b_answer = np.array([[1.44374485, 1.60444091, 0.41649005, 0.71807049]])
        self.assertTrue(np.allclose(dEdW, w_answer), "(FCLayer) Incorrect gradient for weights!")
        self.assertTrue(np.allclose(dEdb, b_answer), "(FCLayer) Incorrect gradient for bias!")        

#### Activation

In [31]:
class TestActivation(unittest.TestCase):
    def setUp(self):
        import activation
        from utils import sigmoid, sigmoid_prime
        self.act = activation.Activation(sigmoid, sigmoid_prime)
        def reset_input():
            self.act.input_data = np.array([[1.38731756, 1.02933563, 1.22931225, 1.42066792],
                                              [3.33225417, 3.75150259, 3.31001342, 4.29398029]])
        self.reset_input = reset_input
        self.delta_n = np.array([[0.5507979 , 0.70814782, 0.29090474, 0.51082761],
                                 [0.89294695, 0.89629309, 0.12558531, 0.20724288]])
        
    def test_forward(self):
        X = np.arange(4).reshape(2, 2)
        output = self.act.forward(X)
        answer = np.array([[0.5       , 0.73105858],
                           [0.88079708, 0.95257413]])
        self.assertTrue(np.allclose(output, answer), "(Activation) forward method failed!")
    
    def test_backward(self):
        self.reset_input()
        output = self.act.backward(self.delta_n)
        answer = np.array([[0.08807356, 0.13733244, 0.05093431, 0.08004899],
                           [0.02972813, 0.02009243, 0.00426841, 0.00275329]])
        self.assertTrue(np.allclose(output, answer), "(Activation) backward method failed!")

#### MSELoss

In [32]:
class TestMSELoss(unittest.TestCase):
    def setUp(self):
        from loss import MSELoss
        self.loss = MSELoss()
        self.target = np.array([3, 9, 8, 9, 8, 7, 2, 7, 8, 5])[:, None]
        self.pred = np.arange(10).reshape(10, 1)

    def test_mse_loss(self):
        output = self.loss.loss(self.pred, self.target)
        answer = 9.85
        self.assertTrue(np.isclose(output, answer), "(MSELoss) Incorrect loss value!")
        
    def test_mse_loss_differentiated(self):
        output = self.loss.diff_loss(self.pred, self.target)
        answer = np.array([-0.3, -0.8, -0.6, -0.6, -0.4, -0.2, 0.4, 0., 0., 0.4])[:, None]
        self.assertTrue(np.allclose(output, answer), "(MSELoss) Incorrect derivative!")

#### CrossEntropyLoss

In [33]:
class TestCrossEntropyLoss(unittest.TestCase):
    def setUp(self):
        from loss import CrossEntropyLoss
        self.loss = CrossEntropyLoss()
        self.pred = np.array([[0.08289002, 0.84846525, 0.10360399, 0.97278462],
                              [0.42144156, 0.2212615 , 0.19189267, 0.94363419],
                              [0.29702955, 0.44302759, 0.09034213, 0.26492577],
                              [0.09610541, 0.37157434, 0.76826065, 0.22563809]])
        self.target = np.array([[0., 1., 0., 0.],
                                [1., 0., 0., 0.],
                                [0., 0., 0., 1.],
                                [0., 0., 1., 0.]])
    
    def test_ce_loss(self):
        output = self.loss.loss(self.pred, self.target)
        answer = 5.000648850895981
        self.assertTrue(np.isclose(output, answer), "(CrossEntropyLoss) Incorrect loss value!")
    
    def test_ce_loss_differentiated(self):
        output = self.loss.diff_loss(self.pred, self.target)
        answer = np.array([[ 0.15137676, -0.67450502,  0.15454507,  0.36858318],
                           [-0.76739833,  0.19040386,  0.18489323,  0.39210124],
                           [ 0.25387967,  0.29378802,  0.20647369, -0.75414138],
                           [ 0.18470979,  0.24329016, -0.63825443,  0.21025447]])
        self.assertTrue(np.allclose(output, answer), "(CrossEntropyLoss) Incorrect derivative!")

#### Nesterov Accelerated Gradient

In [34]:
class TestNesterov(unittest.TestCase):
    def setUp(self):
        from optim import SGD
        from param import Parameter
        self.w = Parameter(value=np.array([[0.1980958 , 0.71222462], [0.5550827 , 0.74202243]]))
        self.w.grad = np.array([[0.14945974, 0.91003691], [0.65230119, 0.56729758]])
        parameters = [self.w]
        self.optimizer = SGD(parameters, lr=0.1, nesterov=True, mu=0.9)
        self.w.velocity = np.array([[0.50626091, 0.34927326], [0.64485252, 0.14303997]])
        
    def test_nag_step(self):
        self.optimizer.step()
        val = self.w.value
        answer = np.array([[0.57976979, 0.82222895],
                           [0.95347602, 0.75009827]])
        self.assertTrue(np.allclose(val, answer), "(SGD) Incorrect Nesterov update!")

In [35]:
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestFullyConnectedLayer('test_forward'))
    suite.addTest(TestFullyConnectedLayer('test_backward'))
    suite.addTest(TestFullyConnectedLayer('test_gradient'))
    suite.addTest(TestActivation('test_forward'))
    suite.addTest(TestActivation('test_backward'))
    suite.addTest(TestMSELoss('test_mse_loss'))
    suite.addTest(TestMSELoss('test_mse_loss_differentiated'))    
    suite.addTest(TestCrossEntropyLoss('test_ce_loss'))
    suite.addTest(TestCrossEntropyLoss('test_ce_loss_differentiated'))
    suite.addTest(TestNesterov('test_nag_step'))
    return suite

In [36]:
if __name__ == "__main__":
    runner = unittest.TextTestRunner(verbosity=3)
    runner.run(suite())

test_forward (__main__.TestFullyConnectedLayer) ... ok
test_backward (__main__.TestFullyConnectedLayer) ... ok
test_gradient (__main__.TestFullyConnectedLayer) ... ok
test_forward (__main__.TestActivation) ... ok
test_backward (__main__.TestActivation) ... ok
test_mse_loss (__main__.TestMSELoss) ... ok
test_mse_loss_differentiated (__main__.TestMSELoss) ... ok
test_ce_loss (__main__.TestCrossEntropyLoss) ... ok
test_ce_loss_differentiated (__main__.TestCrossEntropyLoss) ... ok
test_nag_step (__main__.TestNesterov) ... 

X is ok


ok

----------------------------------------------------------------------
Ran 10 tests in 0.024s

OK
